In [1]:
from pyspark.sql import SparkSession

In [2]:
class FraudDetection():
    def __init__(self):
        self.spark = None
        self.data = None
        
    def create_spark_context(self, ram, rpt=False):
        self.spark = SparkSession.\
            builder.\
            appName("Fraud Detector").\
            master("spark://spark-master:7077").\
            config("spark.executor.memory", "{}g".format(ram)).\
            getOrCreate()
        if rpt: print(self.spark.sparkContext.getConf().getAll())
        return self.spark
    
    def read_file(self, path, rpt=False):
        self.data = self.spark.read.csv(path, header=True, inferSchema=True)
        if rpt: print('number of partitions: {}'.format(self.data.rdd.getNumPartitions()))
        return self.data

In [3]:
detector = FraudDetection()

In [4]:
detector.create_spark_context(8)

In [ ]:
detector.read_file("/opt/workspace/creditcard.csv", True)